In [70]:
from pyTigerGraph import TigerGraphConnection
import pyTigerGraph as tg
import json
import torch
import numpy as np
import pandas as pd
import warnings
import torch
import torch.nn.functional as F
from pyTigerGraph.gds.metrics import Accumulator, Accuracy
import matplotlib.pyplot as plt

c:\Users\ryand\AppData\Local\Programs\Python\Python39\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [71]:
def connection():
    with open("../../configs/tigergraph_config.json", "r") as config:
        args = json.load(config)
    
    conn = tg.TigerGraphConnection(
        host=args["host"],
        graphname=args["graphname"],
        username=args["username"],
        password=args["password"],
        gsqlSecret=args["gsqlSecret"],
        certPath=args["certPath"]
    )

    if not connected(conn):
        raise ConnectionError("Failed to connect to GSQL")
    return conn

def connected(conn):
    if not conn.echo() == "Hello GSQL":
        return False
    return True

In [156]:
conn = connection()

In [106]:
# https://dev.tigergraph.com/forum/t/tigergraph-python-connection-issue/2776
# echo | openssl s_client  -connect 140ae89e8e714a18a7a51df2beca1811.i.tgcloud.io:443 |  sed -ne '/-BEGIN CERTIFICATE-/,/-END CERTIFICATE-/p' > 'C:\Users\ryand\DSC180B\dsc180b-project\my-cert.txt'
# have to put my-cert.txt in C:\User\ryand\.gsql\my-cert.txt
print(conn.gsql("LS"))

---- Global vertices, edges, and all graphs
Vertex Types:
- VERTEX Specialty(PRIMARY_ID id STRING) WITH STATS="OUTDEGREE_BY_EDGETYPE", PRIMARY_ID_AS_ATTRIBUTE="false"
- VERTEX SubSpecialty(PRIMARY_ID id STRING) WITH STATS="OUTDEGREE_BY_EDGETYPE", PRIMARY_ID_AS_ATTRIBUTE="false"
- VERTEX Prescriber(PRIMARY_ID Prescriber_id STRING, pageRank FLOAT, communityId INT, articleRank FLOAT) WITH STATS="OUTDEGREE_BY_EDGETYPE", PRIMARY_ID_AS_ATTRIBUTE="false"
- VERTEX Claim(PRIMARY_ID Claim_id STRING, rx_fill_date DATETIME, ICD10Code STRING, ICD10CodeDescription STRING, CodeGroupTitle STRING) WITH STATS="OUTDEGREE_BY_EDGETYPE", PRIMARY_ID_AS_ATTRIBUTE="false"
- VERTEX Patient(PRIMARY_ID Patient_id STRING) WITH STATS="OUTDEGREE_BY_EDGETYPE", PRIMARY_ID_AS_ATTRIBUTE="false"
Edge Types:
- DIRECTED EDGE submitted_by(FROM Claim, TO Prescriber) WITH REVERSE_EDGE="reverse_submitted_by"
- DIRECTED EDGE reverse_submitted_by(FROM Prescriber, TO Claim) WITH REVERSE_EDGE="submitted_by"
- DIRECTED EDGE associa

In [129]:
with open("../../configs/tigergraph_config.json", "r") as config:
        args = json.load(config)

conn.getToken(args['gsqlSecret'])

('sieph8qrphup7oottriqaajvtqqqfonf', 1677978230, '2023-03-05 01:03:50')

In [75]:
# Number of vertices for every vertex type
conn.getVertexCount('*')

{'User': 243809, 'Comment': 5117970, 'Subreddit': 4420}

In [76]:
# Number of edges for every type
conn.getEdgeCount()

{'wrote': 4701420,
 'reverse_wrote': 4701420,
 'belongs_to': 4701420,
 'reverse_belongs_to': 4701420,
 'replied_to': 2700760,
 'reverse_replied_to': 2700760}

## Feature Engineering

In [157]:
f = conn.gds.featurizer()
# f.installAlgorithm("tg_pagerank")
# f.installAlgorithm("tg_closeness_cent")
# f.installAlgorithm("tg_betweenness_cent")
# f.installAlgorithm("tg_label_prop")
# f.installAlgorithm("tg_maximal_indep_set")

In [136]:
# only combination that works on our current data
tg_pagerank_params = {
  "v_type": "Comment",
  "e_type": "replied_to",
  "result_attr": "pagerank",
}
results = pd.json_normalize(f.runAlgorithm("tg_pagerank",tg_pagerank_params)[0]['@@top_scores_heap'])
results

,Vertex_ID,score
0,c1920o8,41.80026
1,c19z9dl,41.12798
2,c18dyjj,34.89923
3,c18dqrr,32.40516
4,c18gp4r,30.50136
...,...,...
95,c19zapt,10.75795
96,c17jgqc,10.70455
97,c1a2kjk,10.69053
98,c193cb5,10.58636


In [158]:
# only combination that works on our current data
tg_pagerank_params = {
  "v_type": "User",
  "e_type": ["wrote", "reverse_wrote", "replied_to", "reverse_replied_to"],
  "result_attr": "pagerank",
}
results = pd.json_normalize(f.runAlgorithm("tg_reddit_pagerank",tg_pagerank_params)[0]['@@top_scores_heap'])
results

The list of the categories for available algorithms in the GDS (https://github.com/tigergraph/gsql-graph-algorithms):
Centrality: 
 pagerank: 
  global: 
   weighted: 
    Algorithm Name: tg_pagerank_wt
    Algorithm Link: https://github.com/tigergraph/gsql-graph-algorithms/blob/master/algorithms/Centrality/pagerank/global/weighted/tg_pagerank_wt.gsql. 
   unweighted: 
    Algorithm Name: tg_pagerank
    Algorithm Link: https://github.com/tigergraph/gsql-graph-algorithms/blob/master/algorithms/Centrality/pagerank/global/unweighted/tg_pagerank.gsql. 
 article_rank: 
  Algorithm Name: tg_article_rank
  Algorithm Link: https://github.com/tigergraph/gsql-graph-algorithms/blob/master/algorithms/Centrality/article_rank/tg_article_rank.gsql. 
 Betweenness: 
  Algorithm Name: tg_betweenness_cent
  Algorithm Link: https://github.com/tigergraph/gsql-graph-algorithms/blob/master/algorithms/Centrality/betweenness/tg_betweenness_cent.gsql. 
 closeness: 
  approximate: 
   Algorithm Name: tg_closene

ValueError: The query name is not included in the list of queries.

In [153]:
tg_closeness_params = {
    "v_type": "User",
    "e_type": ["replied_to", "reverse_wrote"],
    "reverse_e_type": ["reverse_replied_to"],
    "result_attr": "closeness"
}
results = pd.json_normalize(f.runAlgorithm("tg_closeness_cent", tg_closeness_params))
results
f.runAlgorithm("tg_closeness_cent", tg_closeness_params)

[{'top_scores': []}]

In [147]:
tg_betweenness_params = {
    "v_type": ["Comment"],
    "e_type": ["replied_to"],
    "reverse_e_type": ["reverse_replied_to"],
    "result_attr": "betweenness"
}
results = pd.json_normalize(f.runAlgorithm("tg_betweenness_cent", tg_betweenness_params))
results

c:\Users\ryand\AppData\Local\Programs\Python\Python39\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,top_scores
0,[]


## Data Loader

In [44]:
graph_loader = conn.gds.graphLoader(
    num_batches=1,
    v_extra_feats=["x"],
    e_extra_feats=[],
    output_format = "PyG")

Installing and optimizing queries. It might take a minute if this is the first time you use this loader.
Query installation finished.
